In [1]:
import numpy as np
import pandas as pd

Read data what we download from cafef.vn

# Read and Pre-Processing Data

In [2]:
df = pd.read_csv('data/HSX.csv')

In [3]:
df.columns =['Ticker','Date','Open','High','Low','Close','Volume']

Choose one stock to simulate

In [16]:
tickerID = "DXG"
df_one_stock = df[df['Ticker']==tickerID]
df_one_stock.index = pd.to_datetime(df_one_stock.Date, format='%Y%m%d',infer_datetime_format=True)
df_one_stock=df_one_stock.iloc[::-1]

Check missing and outlier value

In [45]:
df_one_stock.describe()

,Date,Open,High,Low,Close,Volume
count,2.804000e+03,2804.000000,2804.000000,2804.000000,2804.000000,2.804000e+03
mean,2.015175e+07,9.273702,9.444862,9.117063,9.282513,1.641891e+06
std,3.250964e+04,5.888950,6.012956,5.791855,5.908809,2.264010e+06
min,2.009122e+07,1.903900,2.021000,1.903900,1.992100,1.400000e+02
25%,2.012102e+07,4.915425,4.992750,4.809000,4.917100,1.074025e+05
50%,2.015082e+07,7.220000,7.325500,7.094700,7.217100,7.797800e+05
75%,2.018061e+07,12.839275,13.045800,12.622500,12.803050,2.358185e+06
max,2.021032e+07,27.758400,27.967100,27.271400,27.692300,3.315740e+07


# Visualize stock price 
- Using Plotly to plot stock price.
- Vndirect provide the API to get information of specifice stock

In [200]:
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import cufflinks as cf
import requests

In [201]:
df_one_stock_vis = df_one_stock[df_one_stock.index > '2018-01-01'] # just plot stock price from 2018 to now only
# Create figure with secondary y-axis
response = requests.get("https://finfo-api.vndirect.com.vn/stocks?symbol="+tickerID)

fig = make_subplots(specs=[[{"secondary_y": True}]])
# include candlestick with rangeselector
fig.add_trace(go.Candlestick(x=df_one_stock_vis.index,
                open=df_one_stock_vis['Open'], high=df_one_stock_vis['High'],
                low=df_one_stock_vis['Low'], close=df_one_stock_vis['Close'],name='Daily Price'),
               secondary_y=False)
# format chart
fig.update_layout(
        title={
                'text': tickerID+ " - "+response.json()['data'][0]['companyNameEng'],
                'y':0.98,
                'x':0.15,
                'xanchor': 'center',
                'yanchor': 'top'},
        xaxis_title="Date",
        font=dict(
            family="Avenir, monospace",
            size=15,
            color="gray"
        )
    )
# Set y-axes titles
fig.update_yaxes(title_text="Close Price", secondary_y=False,showgrid=False)
fig.update_yaxes(title_text="Volume", secondary_y=True)
fig.update_yaxes(showspikes = True, spikemode = 'across')
fig.update_xaxes(showspikes=True, spikemode= 'across')

# fig.update_layout(paper_bgcolor="#ffffff",plot_bgcolor= '#ffffff')

fig.update_layout(xaxis_rangeslider_visible=False)

# Simulate stock price in the next 30 days using Monte Carlo

In [64]:
nrows = len(df_one_stock)
stock_returns = (df_one_stock['Close'].shift(1)/df_one_stock['Close'])-1

In [203]:
fig = go.Figure([go.Scatter(x=stock_returns.index, y=stock_returns)])
# format chart
fig.update_layout(
        title={
                'text': 'Return of ' + tickerID + ' accross 11 years',
                'y':0.98,
                'x':0.15,
                'xanchor': 'center',
                'yanchor': 'top'}
        )
fig.show()

Simulate 1000 scenarios of stock price for the next 30 days with the current day is 2021/02/05

In [206]:
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# setup
init_notebook_mode(connected=True)
cf.set_config_file(theme='pearl')

fig = simulation_df.iloc[:,:100].iplot(asFigure=True, kind='scatter',xTitle='Number of Day',yTitle='Close Price',
                                       title='Simulate Stock Price in next 30 days')
iplot(fig)